In [1]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

/Users/salvatoredamicis/anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/salvatoredamicis/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input

# Prepare Training set

In [3]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, _ = train_test_split(annotations['mp3_path'], train_size=0.005, test_size=0.001) 

In [4]:
x_train = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)
x_train.shape

100%|██████████| 129/129 [00:00<00:00, 146.86it/s]


(129, 465984)

In [40]:
y_train = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0'])
y_train = y_train.values

In [41]:
model = keras.Sequential()
model.add(Conv1D(filters=128, kernel_size=13, strides=13, input_shape=(x_train.shape[1],1)))
model.add(MaxPool1D(pool_size=13)) #
model.add(Conv1D(filters=256, kernel_size=7, strides=5))
model.add(MaxPool1D(pool_size=7))
model.add(Conv1D(filters=512, kernel_size=5, strides=3))
model.add(MaxPool1D(pool_size=5))
model.add(Conv1D(filters=512, kernel_size=3, strides=1))
model.add(MaxPool1D(pool_size=3))
model.add(Dense(units=30, activation='sigmoid'))

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 35844, 128)        1792      
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 2757, 128)         0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 551, 256)          229632    
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 78, 256)           0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 25, 512)           655872    
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 5, 512)            0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 3, 512)            786944    
__________

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [45]:
model.fit(x_train,y_train,epochs=10)

ValueError: Error when checking input: expected conv1d_29_input to have 3 dimensions, but got array with shape (129, 465984)